In [3]:
def gray_str(n):
    assert n > 0
    if n == 1:
        yield '1'
        yield '0'
        return
    # generator is not reversible
    prev = list(gray_str(n-1))
    for rest in prev:
        yield '1' + rest
    for rest in reversed(prev):
        yield '0' + rest

In [5]:
list(gray_str(3))

['111', '110', '100', '101', '001', '000', '010', '011']

In [13]:
from itertools import cycle, islice

def gray_change(n):
    assert n > 0
    if n == 1:
        yield from cycle((-1, 1))
        
    val = -n
    for x in gray_change(n-1):
        yield x
        yield val
        val = -val

In [14]:
list(islice(gray_change(3), 8))

[-1, -3, -2, 3, 1, -3, 2, 3]

In [16]:
vals = [1, 1, 1]
for i, g in zip(range(8), gray_change(3)):
    # need to be 0-indexed, so abs(g) - 1
    # g // abs(g) = sgn(g)
    vals[abs(g) - 1] += g // abs(g)
    print(i, vals)

0 [0, 1, 1]
1 [0, 1, 0]
2 [0, 0, 0]
3 [0, 0, 1]
4 [1, 0, 1]
5 [1, 0, 0]
6 [1, 1, 0]
7 [1, 1, 1]


In [19]:
def gray_change_finite(n):
    assert n > 0
    if n == 1:
        yield -1
        yield 1
        return        
    val = -n
    for x in gray_change_finite(n-1):
        yield x
        yield val
        val = -val

In [20]:
list(gray_change_finite(4))

[-1, -4, -3, 4, -2, -4, 3, 4, 1, -4, -3, 4, 2, -4, 3, 4]

In [26]:
def gray_by_changes(n):
    assert n > 0
    vals = [1] * n
    for g in gray_change_finite(n):
        vals[abs(g) - 1] += g // abs(g)
        yield vals[:]

In [27]:
list(gray_by_changes(4))

[[0, 1, 1, 1],
 [0, 1, 1, 0],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 0, 0, 1],
 [0, 0, 0, 0],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [1, 0, 1, 1],
 [1, 0, 1, 0],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 1, 0, 1],
 [1, 1, 0, 0],
 [1, 1, 1, 0],
 [1, 1, 1, 1]]

In [29]:
list(gray_str(4))

['1111',
 '1110',
 '1100',
 '1101',
 '1001',
 '1000',
 '1010',
 '1011',
 '0011',
 '0010',
 '0000',
 '0001',
 '0101',
 '0100',
 '0110',
 '0111']

Now let's stop here. I already thought my recursive generator algorithm was wrong, but it simply generates the sequence in a bit different order.

Now let me look to the spoilers, erm, cheat sheet, I mean, the professional reference ("Hacker's Delight" by Warren). I expect to find there one of two things - either a closed form formula (which would allow to produce the elements in parallel - vectorized), or at least some iterative hack - some mask for xoring the last element - not parallelizable, but wouldn't need a deep stack like my recurrences...

### After the reading

OK, actually there is a lot of gray codes, something like $2^n n!$ for $n$ bit words. But some are not cyclical, and some are not reflective.

The one that starts with `0000` and ends with `1000` is called reflective binary Gray code, and it maps nicely from ints:

$ G = B \oplus (B \gg 1) $

The inverse is quite crazy, because we need to XOR together all the shifts!

$ B = \bigoplus\limits_{i=0}^{n-1} (G \gg i) $

(just found out that `\oplus` doesn't work with `\limits`, but `\bigoplus` does...)



In [36]:
# inventing hack for zero-padding:
'{:0{}d}'.format(int(bin(10)[2:]), 8)

'00001010'

In [48]:
def gray(n, bits=4):
    assert n >= 0
    value = n ^ (n >> 1)
    if bits is None:
        return value
    # return n, value, bin(value)[2:]
    return '{:0{}d}'.format(int(bin(value)[2:]), bits)

In [49]:
[gray(i) for i in range(16)]

['0000',
 '0001',
 '0011',
 '0010',
 '0110',
 '0111',
 '0101',
 '0100',
 '1100',
 '1101',
 '1111',
 '1110',
 '1010',
 '1011',
 '1001',
 '1000']

### Further reading

Well, there are some obscure algorithms e.g. to increment a number while in Gray code (without decoding to binary and back).

There are also non-binary Gray codes, $(n, k)$-gray code, e.g. $(10, 3)$ are 3-digit decimals. They would go from `000` to `999`,
by changing digits by $1$ between successive elements? Not sure if this can be made cyclical really.


Why? Well because in binary numbers, the basic 1-digit number is itself cyclical: $0, 1, 0, 1, \dots$
Can this be done for $0, 1, 2, \dots$? It doesn't seem so.

Let's have a look: $00, 01, 02, 12, 11, 10$ - this would be cyclical, up till here, but now there comes $20, 21, 22$ to complete
the code for all the $ 2^3 = 9 $ two digit ternary numbers.